In [1]:
import os

In [2]:
%pwd

'd:\\My\\AboutData\\Project\\Weather-Prediction\\notebooks'

In [3]:
os.chdir('../')
%pwd

'd:\\My\\AboutData\\Project\\Weather-Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    transformed_filename: Path

In [6]:
from src.constants import *
from src.utils.helper import read_yaml, create_directories

In [7]:
class ConfigurationManager():
    def __init__(self, config_filepath = CONFIG_FILEPATH, params_filepath = PARAMS_FILEPATH, schema_filepath = SCHEMA_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            transformed_filename = config.transformed_filename
        )

        return data_transformation_config


In [8]:
import os
from src import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def handle_missing_values(self):
        data = pd.read_csv(self.config.data_path)
        data.dropna(subset=['weather_code'], inplace=True)
        data.to_csv(os.path.join(self.config.root_dir, self.config.transformed_filename))

        logger.info("Missing value handled successfully")

    def train_test_spliting(self):
        data = pd.read_csv(os.path.join(self.config.root_dir, self.config.transformed_filename))

        train, test = train_test_split(data, test_size=.25)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.handle_missing_values()
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-05-22 14:02:52,239: INFO: helper: yaml file: config\config.yaml loaded successfully]
[2025-05-22 14:02:52,323: INFO: helper: yaml file: params.yaml loaded successfully]
[2025-05-22 14:02:52,328: INFO: helper: yaml file: schema.yaml loaded successfully]
[2025-05-22 14:02:52,331: INFO: helper: created directory at: artifacts]
[2025-05-22 14:02:52,333: INFO: helper: created directory at: artifacts/data_transformation]
[2025-05-22 14:02:52,426: INFO: 1547636484: Missing value handled successfully]
[2025-05-22 14:02:52,489: INFO: 1547636484: Splited data into training and test sets]
[2025-05-22 14:02:52,491: INFO: 1547636484: Train shape: (65, 15)]
[2025-05-22 14:02:52,493: INFO: 1547636484: Test shape: (22, 15)]


## Data Transformation Testing